# Exam Planning-Lab

## Exercise 1

Consider the following environment:

<img src="images/env1.png" alt="ex1" style="zoom: 40%;" />

The agent starts in cell $(0, 0)$ and has to reach the treasure in $(2, 3)$. In addition to the walls of the previous environments, the floor is covered with lava, however the agent can resist to high temperature and it can use the heat to recharge its batteries, hence it receives a positive reward for being on a lava cell. The environment is deterministic and the agent must avoid the black pits of death (cells $(0,3)$, $(1, 3)$, $(1,1)$). 
Assume that you do not have access to the motion model and to reward and that the problem is undiscounted (i.e., $\gamma$=1)


In [2]:
import os, sys
module_path = os.path.abspath(os.path.join('tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import tools.gym, tools.envs
from tools.utils.ai_lab_functions import *
from timeit import default_timer as timer
from tqdm import tqdm as tqdm
import numpy as np

def epsilon_greedy(q, state, epsilon):
    """
    Epsilon-greedy action selection function
    
    Args:
        q: q table
        state: agent's current state
        epsilon: epsilon parameter
    
    Returns:
        action id
    """
    an = q.shape[1]
    probs = np.full(an, epsilon / an)
    probs[q[state].argmax()] += 1 - epsilon
    return np.random.choice(an, p=probs)


env = tools.gym.make('LavaFloor-v0')
env.render()

[['S' 'L' 'L' 'P']
 ['L' 'P' 'L' 'P']
 ['L' 'L' 'L' 'G']]


### 1.1) Given the environment reported above, find a policy with a suitable algorithm of your choice. Print the resulting policy (you can use the provided code for this)
In this particular case since the agent doesnt need to many requirements
Q-learning requires less to guarantee convergence, but if I say that I want a safer policy and go on with sarsa its fine

In [3]:
def sarsa(environment, episodes, alpha, gamma, expl_func, expl_param):
    """
    Performs the SARSA algorithm for a specific environment
    
    Args:
        environment: OpenAI gym environment
        episodes: number of episodes for training
        alpha: alpha parameter
        gamma: gamma parameter
        expl_func: exploration function (epsilon_greedy, softmax)
        expl_param: exploration parameter (epsilon, T)
    
    Returns:
        (policy, rewards, lengths): final policy, rewards for each episode [array], length of each episode [array]
    """

    q = np.zeros((environment.observation_space.n, environment.action_space.n))
    rewards = np.zeros(episodes)
    lengths = np.zeros(episodes)
    
    
    for episode in range(episodes):
        s = environment.reset()
        a = expl_func(q,s,expl_param)

        step = 1
        while step is not None:
            step = environment.step(a)
            if step is None:
                break
            next_state, reward, _, _ = step
            a_1 = expl_func(q,next_state,expl_param)

            q[s][a] = q[s][a] + alpha*(reward + gamma*(q[next_state][a_1] - q[s][a]))
            s = next_state
            a = a_1
            
            if s == environment.goalstate:
                break

        rewards[episode] += reward
        lengths[episode] += episode
    
    policy = [0 for _ in range(environment.observation_space.n)]
    for s in range(environment.observation_space.n):
        for a in environment.actions:
            policy[s] += np.argmax(q[s][a])


    policy = q.argmax(axis=1)
    return policy, rewards, lengths


In [4]:
episodes = 1000
gamma = 1
alpha = .3
epsilon = .1


env = tools.gym.make('LavaFloor-v0')
env.render()
print() 

policy, rewards, lengths = sarsa(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
print("Policy:\n {} \n".format(np.vectorize(env.actions.get)(policy.reshape(env.shape))))

[['S' 'L' 'L' 'P']
 ['L' 'P' 'L' 'P']
 ['L' 'L' 'L' 'G']]

Policy:
 [['U' 'L' 'L' 'L']
 ['U' 'L' 'D' 'L']
 ['L' 'L' 'L' 'L']] 



# We notice that is mostly trying to avoid the pit than going towards the goal! (THATS WHY)

### 1.2) Find a way to force the agent to choose the shortest path towards the goal. You can change these parameters: episodes, alpha, gamma exploration function and exploration parameter. 

##### Hint: you should tune a parameter to consider the immediate reward rather than the long-term one... 

DECREASE GAMMA! REACHES THE TREASURE AS SOON AS POSSIBLE

In [5]:
episodes = 200
gamma =  .1
alpha =  .3
epsilon = .1

env.render()
print() 

# Q-Learning or SARSA epsilon greedy
env = tools.gym.make('LavaFloor-v0')
p, r, l = sarsa(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
print("Policy:\n {} \n".format(np.vectorize(env.actions.get)(p.reshape(env.shape))))

[['S' 'L' 'L' 'P']
 ['L' 'P' 'L' 'P']
 ['L' 'L' 'L' 'G']]

Policy:
 [['L' 'L' 'L' 'L']
 ['U' 'L' 'R' 'L']
 ['U' 'L' 'L' 'L']] 



### 1.3) Consider and environment with states {A, B, C, D}, actions {r, l} where states {A, D} are terminal. Consider the following sequence of learning episodes:
* E1: (B, r, C, −1),(C, r, C, −1),(C, r, D, +1)
* E2: (B, r, C, −1),(C, r, D, +1)
* E3: (B, l, A, +5)
* E4: (B, l, B, −1),(B, l, B, −1),(B, l, A, +5)
### Compute v(s) for all non-terminal states by using a sample-based evaluation approach (i.e., computing the values with the function reported below). Assume $\alpha$ = .5 and $\gamma$ = 1.
### $V(s) = (1- \alpha) \cdot V(s) + \alpha \cdot (r + \gamma \cdot V(s'))$ 
### where $s$ is the state under consideration (first element of each tuple), $s'$  (third element of each tuple ) the state reached by starting from $s$ and performing the action $a$ (second element of each tuple). And finally, $r$ is the reward (last element of each tuple).

In [6]:
episodes = {1: [('B', 'r', 'C', -1), ('C', 'r', 'C', -1),('C', 'r', 'D', 1)], 
            2: [('B', 'r', 'C', -1), ('C', 'r', 'D', 1)], 
            3: [('B', 'l', 'A', 5)], 
            4: [('B', 'l', 'B', -1),('B', 'l', 'B', -1),('B', 'l', 'A', 5)]}

v = {'A': 5, 'B': 0, 'C': 0, 'D': 1}
alpha = 0.5
gamma = 1


for episode, values in episodes.items():
    for s,a,s_1,r in values:
        v[s] = (1 - alpha)*v[s] + alpha*(r + gamma*(v[s_1]))

print(v)

{'A': 5, 'B': 6.90625, 'C': 1.375, 'D': 1}


## Exercise 2

Consider the figure below where **S=$(1,2)$** and **G=$(3,1)$** are the starting and goal positions respectively. Consider the problem of finding a minimum cost path from S to G assuming the agent can move in the four directions (if there is no
obstacle) and that each movement has a unitary cost. The environment is deterministic. Answer the following questions:

<img src="images/ex2.png" alt="ex2" style="zoom: 40%;" />


In [7]:
env = tools.gym.make('SmallMaze-v0')
env.render()

[['C' 'C' 'C' 'C' 'C']
 ['C' 'C' 'S' 'W' 'C']
 ['C' 'W' 'W' 'W' 'C']
 ['C' 'G' 'C' 'C' 'C']]


### 2.1) Verify by using the code developed in the lab that the Manhattan distance (l1_norm) is a  *consistent* heuristic in this environment. In particular, you should implement a function that checks whether for every couple of state $(s,s')$ (where $s'$ is a successor state of $s$), the consistency condition is verified. The function should return true if the heuristic is consistent and false otherwise. Recall that every action has a cost of 1...


The best way to solve the excercise is to just say why I know that heuristic is consistent and then to code the consistency proof: I have to check that every state 

In [8]:
def consistency_proof(goalpos,s,s_1):
    if(Heu.l1_norm(s,goalpos) + 1) >= Heu.l1_norm(s_1,goalpos): #consistency: h(i) <= c(i,i+1) + h(i + 1)
        return True
    return False

In [30]:
def check_heuristic(env):
    goalpos = env.state_to_pos(env.goalstate)
    for s in range(env.observation_space.n):

        if env.grid[s] == "W":
            continue
        for action in range(env.action_space.n):
            s_pos = env.state_to_pos(s)
            s_1_pos = env.state_to_pos(env.sample(s,action))
            if(not consistency_proof(goalpos,s_pos,s_1_pos)):
                print(f"The heuristic is not consistent for {s} and {s_1}")
                return False
    print("The heuristic is consistent!")
    return True

In [31]:
goalpos = env.state_to_pos(env.goalstate)
print(check_heuristic(env))

The heuristic is consistent!
True


### 2.2) Consider the A* algorithm and assume want to achieve optimality. Based on the *consistent* heuristics of Section 2.1, state whether it is best to use a graph search or tree search strategy. Motivate your answer and show the results of A* execution and the differences between the two versions in terms of the computed solution, number of nodes generated and maximum number of nodes in memory. 


Since we have a consistent heuristic we have to choose a graph search version to achieve optimality. We know only if tree search + admissible heuristic or graph search + consistent heuristic => optimality of A*. In all other situations, we cannot guarantee optimality.

In [11]:
def present_with_higher_cost(queue, node):
    if node.state in queue:
        if queue[node.state].value > node.value: 
            return True
    return False

In [12]:
def astar_tree_search(environment):
    """
    A* Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    
    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    startpos = env.state_to_pos(env.startstate)
    queue.add(Node(environment.startstate,None,1,Heu.l1_norm(startpos,goalpos)+1))
    
    time_cost = 1
    space_cost = 1
    
    while True:
        if queue.is_empty(): 
            return None
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
        
        # Look around
        for action in range(environment.action_space.n):
            #added path cost that's an heuristic
            child_state = environment.sample(node.state, action)
            child_pos = env.state_to_pos(child_state)
            child = Node(child_state, node, node.pathcost + 1, Heu.l1_norm(child_pos, goalpos) + node.pathcost +1)
            time_cost += 1
            
            queue.add(child)
                
        space_cost = max(space_cost, len(queue))

In [13]:
def astar_graph_search(environment):
    """
    A* Graph Search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    startpos = env.state_to_pos(env.startstate)
    queue.add(Node(environment.startstate,None,1,Heu.l1_norm(startpos,goalpos)+1))
    
    
    explored = set()
    time_cost = 1
    space_cost = 1
    
    while True:
        if queue.is_empty(): 
            return None
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
        
        explored.add(node.state)
        
        # Look around
        for action in range(environment.action_space.n):
            #added path cost that's an heuristic
            child_state = environment.sample(node.state, action)
            child_pos = env.state_to_pos(child_state)
            child = Node(child_state, node, node.pathcost + 1, Heu.l1_norm(child_pos, goalpos) + node.pathcost + 1) 
            time_cost += 1
            
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                
            elif present_with_higher_cost(queue, child):
                queue.replace(child)
                
        space_cost = max(space_cost, len(queue) + len(explored))

In [14]:
env = tools.gym.make('SmallMaze-v0')
solution_ts, time_ts, memory_ts = astar_tree_search(env)
solution_gs, time_gs, memory_gs = astar_graph_search(env)

print("Solution (tree-search): {}".format(solution_2_string(solution_ts, env)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}\n".format(memory_ts))
print('='*65)
print("\nSolution (graph-search): {}".format(solution_2_string(solution_gs, env)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}\n".format(memory_gs))

Solution (tree-search): [(1, 1), (1, 0), (2, 0), (3, 0), (3, 1)]
N° of nodes explored: 125
Max n° of nodes in memory: 94


Solution (graph-search): [(1, 1), (1, 0), (2, 0), (3, 0), (3, 1)]
N° of nodes explored: 29
Max n° of nodes in memory: 10



### 2.3) Let us consider BFS, show the path of the optimal solution (avoiding the repetition of states) to achieve the goal. In this scenario can we guarantee that the returned solution is the least cost one? If we used Iterative Deepening Search(IDS) with the same methodology used for BFS can we guarantee a least cost solution? Justify your answer and show the results of the different strategies by using the code developed during the lab. 


In [15]:
from sre_constants import FAILURE
from importlib_metadata import NullFinder


def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    
    #initial test of node
    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost

    #initialize frontier and explored
    frontier = NodeQueue()
    frontier.add(node)
    explored = []

    while not frontier.is_empty():
        node = frontier.remove()
        explored.append(node.state)
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state,action), node)
            time_cost += 1
            if child.state not in explored and child.state not in frontier:
                if child.state == problem.goalstate:
                   return build_path(child), time_cost, space_cost    
                frontier.add(child)
                if(len(frontier) > space_cost or len(explored) > space_cost):
                    space_cost += max(len(frontier), len(explored))
    return FAILURE, time_cost, space_cost

In [16]:
env = tools.gym.make('SmallMaze-v0')
solution_gs, time_gs, memory_gs = BFS_GraphSearch(env)
print("Solution (BFS graph-search): {}".format(solution_2_string(solution_gs, env)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}\n".format(memory_gs))

Solution (BFS graph-search): [(1, 1), (1, 0), (2, 0), (3, 0), (3, 1)]
N° of nodes explored: 39
Max n° of nodes in memory: 15



In [17]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    node = Node(problem.startstate, None)
    return RDLS_Function(node, problem, limit, set())

In [18]:
def Recursive_DLS_GraphSearch(node, problem, limit, explored):
    """
    Recursive DLS
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'limit
        explored: completely explored nodes
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    #initialize frontier and explored
    space_cost = node.depthcost
    time_cost = 1 
    explored.add(node.state)

    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost
    elif limit == 0:
        return "cut_off", time_cost, space_cost
    else:
        cut_off_occured = False
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state,action), node)
            
            if child.state not in explored:
                result, time_cost_child, space_cost_child = Recursive_DLS_GraphSearch(child, problem, limit - 1, explored)

                space_cost = max(space_cost, space_cost_child)
                time_cost += time_cost_child
                
                if result == "cut_off":
                    cut_off_occured = True
                elif result != FAILURE:
                    return result, time_cost, space_cost
                
        if cut_off_occured:
            return "cut_off", time_cost, space_cost 
        else:
            return FAILURE, time_cost, space_cost

In [19]:
#example IDS+GRAPH SEARCH
def IDS(problem, DLS_Function):
    """
    Iteartive_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    total_time_cost = 0
    total_space_cost = 1
    
    for i in zero_to_infinity():
        result, time_cost, space_cost = DLS(problem,i,DLS_Function)
        total_time_cost += time_cost
        total_space_cost = max(space_cost, total_space_cost)
        if result!="cut_off":
            return result, total_time_cost, total_space_cost, i

In [20]:
env = tools.gym.make('SmallMaze-v0')
solution_gs, time_gs, memory_gs, iterations_gs = IDS(env, Recursive_DLS_GraphSearch)
print("Solution (IDS graph-search): {}".format(solution_2_string(solution_gs, env)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}\n".format(memory_gs))

Solution (IDS graph-search): [(1, 1), (1, 0), (2, 0), (3, 0), (3, 1)]
N° of nodes explored: 37
Max n° of nodes in memory: 5



### Discuss the results achieved with respect to the 2.3 question:

No, we cannot guarantee that BFS returns the least cost solution (used graph search to avoid repetition of states). If we unluckily explore the wrong node first, with something already being in the frontier we won't find the optimal solution.\
Graph search version of IDS doesn't guarantee optimality either: if we use tree search it will guarantee optimality.

### WRONG FIRST ANSWER
In this case we can guarantee with BFS because the coast are all uniform (its like UCS) so it guarantees to find the optimal solution. This is not generally guranteed though!\
In this case it doesnt matter if its graph or tree search

### CORRECT SECOND ANSWER BUT
This gives us the optimal solution in this case: BUT ITS NOT GUARANTEE!\
Graph search version of IDS doesn't guarantee optimality either: if we use tree search it will guarantee optimality.

### A*

Its optimal efficient: guarantees optimality with the less node in memory

## Exercise 3

Consider the environment displayed in Figure below, where states $(0, 3)$ and $(1, 3)$ are terminal states with reward $+1$ and $−1$ respectively. The agent can move in the four directions. The transition model states that for every state and action the agent has $0.8$ chances of moving in the chosen direction and $0.1$ chances to move in the othogonal directions.The reward model states that for every state, action and successor state the agent pays $−0.04$. Assume that the dicount factor is set to $0.9$. Answer the following questions: 

<img src="images/ex3.1.png" alt="ex3" style="zoom: 40%;" />


In [21]:
env = tools.gym.make('VeryBadLavaFloor-v0')
env.render()

[['S' 'L' 'L' 'G']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'L']]


### 3.1) Use one of the methods developed in the lab to compute the value function for the left diagram. 

Value iteration is the best because we are not actually asking for a policy but for the values

In [22]:
def value_iteration(environment, maxiters=300, discount=0.9, max_error=1e-3):
    """
    Performs the value iteration algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        maxiters: timeout for the iterations
        discount: gamma value, the discount factor for the Bellman equation
        max_error: the maximum error allowd in the utility of any state
        
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """
    delta = 0 # maximum change in the utility o any state in an iteration
    U_1 = [0 for _ in range(environment.observation_space.n)] # vector of utilities for states S
    iters = 0
    U = U_1.copy()
    
    while True:

        delta = 0
        U = U_1.copy()
        
        for s in range(environment.observation_space.n):

            action_value = [0 for _ in range(environment.action_space.n)]
            
            for a in range(environment.action_space.n):
                for s_1 in range(environment.observation_space.n):
                    action_value[a] += environment.T[s,a,s_1]*U[s_1]

            

            if environment.grid[s] == 'G' or environment.grid[s] == 'P':
                 U_1[s] = environment.RS[s]
            else:
                U_1[s] = environment.RS[s] + discount*max(action_value)

            if abs(U_1[s] - U[s]) > delta: 
                delta = U_1[s] - U[s]

        iters += 1

        if(delta < max_error * (1 - discount)/discount or iters >= maxiters):
            break

    return values_to_policy(np.asarray(U), environment),np.asarray(U) # automatically convert the value matrix U to a policy

In [23]:
env = tools.gym.make('VeryBadLavaFloor-v0')
policy, values = value_iteration(env)
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
print(values)
print(policy_render)

[ 0.50939438  0.64958568  0.79536209  1.          0.39844322  0.
  0.48644002 -1.          0.29628832  0.253867    0.34475423  0.12987275]
[['R' 'R' 'R' 'L']
 ['U' 'L' 'U' 'L']
 ['U' 'R' 'U' 'L']]


### Results:

<img src="images/ex3.png" alt="ex3" style="zoom: 40%;" />

### 3.2) Consider the right diagram and focus on states $(2, 1)$. State whether the action reported in the right diagram (the blue one) represents the optimal action for that state. Motivate your answer with the code...

In [24]:
actions = {0: "L", 1: "R", 2: "U", 3: "D"}

value_function = [0.50939438, 0.64958568, 0.79536209, 1, 0.39844322, 0, 0.48644002, -1, 0.29628832,  0.253867, 0.34475423, 0.12987275]
id_start_state = 9
gamma = 0.9


values_ex = [0, 0, 0, 0]
'''
YOUR CODE HERE
'''
    
print(np.asarray(values_ex))
print(f'The correct action to perform should be: {actions[np.argmax(values_ex)]}')

[0 0 0 0]
The correct action to perform should be: L


### 3.3) Compute the probability of ending in state (1, 3) if we execute the sequence of actons < Up, Up > from state (2, 2). Motivate your answer reporting the code and the solution printed. The following image shows a diagram to guide the solution process as a hint for you:


<img src="images/example.png" alt="ex3" style="zoom: 30%;" />



In [25]:
id_start_state = 10
state = id_start_state
actions = {0: "L", 1: "R", 2: "U", 3: "D"}

prob = 0 #probability of the action that end in pit
action = 2
probs_fin = 0 #sum of probability

for next_state in range(env.observation_space.n):
    
    if env.T[state, action, next_state] == 0:
        continue


    probs = env.T[state, action, next_state]

    for second_next_state in range(env.observation_space.n):
        
        if env.T[next_state, action, second_next_state] == 0:
            continue

        if env.grid[second_next_state] == "P": #check that the state is (1,3) i.e., the black pit
            
            probs *= env.T[next_state, action, second_next_state]

            print(f'{env.state_to_pos(state)} --> {env.state_to_pos(next_state)} --> {env.state_to_pos(second_next_state)}')
            print(f'probs: {env.T[state, action, next_state]} --> {env.T[next_state, action, second_next_state]}')

            probs_fin += probs

            print('================')
            break
    
print()
print('Probability: ', probs_fin)  

(2, 2) --> (1, 2) --> (1, 3)
probs: 0.8 --> 0.1
(2, 2) --> (2, 3) --> (1, 3)
probs: 0.1 --> 0.8

Probability:  0.16000000000000003


### 3.4) Consider the following environment where states $(0, 3)$ and $(1, 3)$ are terminal states with reward $+1$ and $−1$ respectively. Assume the transition model is the same one defined above and that the discounted factor is $0.9$ as above. However, now the agent pays $−0.4$ instead of $−0.04$ for every action, state and successor state.  Compute the new value function and the optimal policy for this new environment. 


<img src="images/env_3.4.png" alt="ex3" style="zoom: 40%;" />

In [26]:
env = tools.gym.make('NiceLavaFloor-v0')
policy, values = value_iteration(env)
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
print(values)
print(policy_render)

[-1.084   -0.35824  0.27392  1.      -1.084    0.      -0.37984 -1.
 -1.084   -1.084   -1.084   -1.084  ]
[['R' 'R' 'R' 'L']
 ['L' 'L' 'U' 'L']
 ['L' 'L' 'U' 'U']]


### 3.5) Modify the value iteration parameters so that the policy allows the agent to reach the goal only starting from states $(0,1)$, $(0,2)$ and $(1,2)$, as reported in the image below . Motivate your answer.


#### hint: given the negative reward the agent should care about the immediate reward and not the long-term reward.



<img src="images/result_ex3.5.png" alt="ex3" style="zoom: 40%;" />

In [27]:
env = tools.gym.make('NiceLavaFloor-v0')
policy, values =  value_iteration(env, discount=0.1)
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
print(values)
print(policy_render)

[-0.44  -0.44  -0.328  1.    -0.44   0.    -0.44  -1.    -0.44  -0.44
 -0.44  -0.44 ]
[['L' 'R' 'R' 'L']
 ['L' 'L' 'U' 'L']
 ['L' 'L' 'L' 'D']]
